In [1]:
import sqlite3

database_path = 'Data//corpus_base.db'

conn = sqlite3.connect(database_path)

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

## Создание базы

In [2]:
cursor.execute('CREATE TABLE Sentences (id INTEGER PRIMARY KEY, sentenceText TEXT)')
cursor.execute('CREATE TABLE Words (word VARCHAR(50) PRIMARY KEY, mentions TEXT)')

## Заполнение базы

**Примечание**: Если к базе установлено несколько соединений и одно из них осуществляет модификацю базы, то база SQLite залочивается до завершения (метод соединения **.commit()**) или отмены (метод соединения **.rollback()**) транзакции.

In [3]:
sentencesList = list()
inputFile = 'Data//corpus.txt'
file_obj = open(inputFile, 'r', encoding="utf8")
for line in file_obj:
    sentencesList.append(line)
file_obj.close()
print(len(sentencesList))

197816


In [4]:
for index, sentence in enumerate(sentencesList):
    cursor.execute("INSERT into Sentences values(:index, :sentence)", {"index": index + 1, "sentence": sentence})

# Если мы не просто читаем, но и вносим изменения в базу данных - необходимо сохранить транзакцию
conn.commit()

In [5]:
wordsDict = dict()
for index, sentence in enumerate(sentencesList):
    words = sentence.split(' ')[1:]
    words = words[:-1]
    for word in words:
        if word not in wordsDict:
            wordsDict[word] = list()
        wordsDict[word].append(str(index + 1))

In [8]:
for word in wordsDict:
    cursor.execute("INSERT into Words values(:word, :mentions)", {"word": word, "mentions": ' '.join(wordsDict[word])})
conn.commit()

## Чтение из базы

In [6]:
trainList = ['балка',
             'вид',
             'винт',
             'горн',
             'губа',
             'жаба',
             'клетка',
             'крыло',
             'купюра', 
             'курица', 
             'лавка', 
             'лайка', 
             'лев', 
             'лира', 
             'мина', 
             'мишень', 
             'обед', 
             'оклад', 
             'опушка', 
             'полис', 
             'пост', 
             'поток', 
             'проказа', 
             'пропасть', 
             'проспект', 
             'пытка',
             'рысь',
             'среда',
             'хвост',
             'штамп',
            ]
#trainDict = dict()

for trainWord in trainList:
    sentencesList = list()
    # Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
    cursor.execute("SELECT word, mentions FROM words WHERE word = :word", {"word": trainWord})

    # Получаем результат сделанного запроса
    result = cursor.fetchall()
    try:
        wordTuple = result[0]
    except:
        print('ERROR', trainWord)
        continue
    word = wordTuple[0]
    mentions = wordTuple[1].split(' ')
    mentions = [int(x) for x in mentions]
    #trainDict[word] = mentions
    
    for mention in mentions:
        cursor.execute("SELECT id, sentenceText FROM Sentences WHERE id = :mention", {"mention": mention})
        sentenceTuple = cursor.fetchall()[0]
        sentenceId = sentenceTuple[0]
        sentenceText = sentenceTuple[1]
        sentencesList.append('\t' + sentenceText)
    
    outputFile = 'Trainset//bts-rnc//' + trainWord + '.txt'
    file_obj = open(outputFile, 'w', encoding="utf8")
    file_obj.writelines(sentencesList)
    file_obj.close()

ERROR горн


In [10]:
trainDict['замок']

[528,
 563,
 1370,
 1378,
 1400,
 1416,
 9421,
 11546,
 23599,
 24093,
 24095,
 41813,
 42298,
 43843,
 44441,
 44508,
 44601,
 44733,
 48079,
 49530,
 51247,
 51248,
 51249,
 51251,
 51266,
 51267,
 52570,
 53537,
 53598,
 54799,
 54801,
 54803,
 54804,
 54805,
 54807,
 54808,
 54809,
 55980,
 55981,
 55982,
 55983,
 55984,
 55986,
 55988,
 55991,
 56022,
 56022,
 57042,
 58962,
 58963,
 59489,
 59491,
 59492,
 59493,
 63182,
 63611,
 70763,
 70829,
 74612,
 82136,
 82143,
 89369,
 91696,
 94719,
 99857,
 102287,
 109718,
 112492,
 112703,
 113267,
 116671,
 117323,
 117623,
 121174,
 121186,
 125670,
 126322,
 126622,
 130173,
 130185,
 132388,
 134108,
 137921,
 138349,
 139452,
 139609,
 140427,
 141697,
 143374,
 149161,
 149860,
 150929,
 151857,
 154392,
 157419,
 160751,
 162726,
 164649,
 164975,
 172316,
 175976,
 179325,
 180045,
 181718,
 182103,
 182104,
 185220,
 185519,
 189041,
 189633,
 190262,
 190262,
 190263,
 191766,
 191780,
 192313]

In [8]:
# Не забываем закрыть соединение с базой данных
conn.close()